In [1]:
import pyspark

In [2]:
sc = pyspark.SparkContext()

## 利用ArcGIS的arcpy包进行序列化，要求是你的Python环境使用的是ArcGIS的Python

In [3]:
import arcpy

In [4]:
feat = arcpy.FeatureSet("./data/dltb.shp")

## 利用arcpy的da包，将数据读取成一个元祖，由几何对象和地类名称两个字段组成

In [5]:
featlist = [(row[0],row[1]) \
            for row in arcpy.da.SearchCursor(feat,["SHAPE@","DLMC"])]

In [6]:
featlist

[(<Polygon object at 0x269e97b4588[0x269e97a99b8]>, '旱地'),
 (<Polygon object at 0x269e97b45c0[0x269e97a9a08]>, '园地'),
 (<Polygon object at 0x269e97b45f8[0x269e97a9a58]>, '林地'),
 (<Polygon object at 0x269e97b4630[0x269e97a9aa8]>, '园地'),
 (<Polygon object at 0x269e97b4668[0x269e97a9b20]>, '园地'),
 (<Polygon object at 0x269e97b46a0[0x269e97a9be8]>, '园地'),
 (<Polygon object at 0x269e97b46d8[0x269e97a9c38]>, '园地'),
 (<Polygon object at 0x269e97b4710[0x269e97a9c88]>, '园地'),
 (<Polygon object at 0x269e97b4748[0x269e97a9cd8]>, '旱地'),
 (<Polygon object at 0x269e97b4780[0x269e97a9d50]>, '林地'),
 (<Polygon object at 0x269e97b47b8[0x269e97a9da0]>, '园地'),
 (<Polygon object at 0x269e97b47f0[0x269e97a9e18]>, '园地'),
 (<Polygon object at 0x269e97b4828[0x269e97a9eb8]>, '林地'),
 (<Polygon object at 0x269e97b4860[0x269e97a9f08]>, '林地'),
 (<Polygon object at 0x269e97b4898[0x269e97a9f80]>, '林地'),
 (<Polygon object at 0x269e97b48d0[0x269e97a9fa8]>, '园地'),
 (<Polygon object at 0x269e97b4908[0x269e97a9ad0]>, '园地'

## 将对象直接转换为PySpark的RDD

In [7]:
frdd = sc.parallelize(featlist)

## 计数

In [8]:
frdd.count()

192

## 按照地类名称进行面积统计

### 其中，f[0]是一个polygon对象，area是用来计算对象的面积的属性

In [9]:
frdd.map(lambda f : (f[1],f[0].area)).reduceByKey(lambda x,y:(x+y)).collect()

[('林地', 4250805.430885682),
 ('旱地', 1062114.5405802839),
 ('园地', 974189.4862168157),
 ('其他独立建设用地', 257737.88656116943),
 ('农村居民点用地', 39557.913537067856)]